In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
import os
import wandb
from pytorch_lightning.loggers import WandbLogger
from opf.utils import model_from_parameters

%aimport opf.utils
%aimport opf.modules
%aimport opf.powerflow

In [3]:
params = dict(
    case_name="case30",
    adj_scaling="auto",
    adj_threshold=0.01,
    batch_size=256,
    max_epochs=1,
    patience=1000,
    K=8,
    F=32,
    L=2,
    s=10,
    t=500,
    F_MLP=4,
    L_MLP=1,
    cost_weight=0.01,
    lr=1e-4,
    eps=1e-4,
    constraint_features=False,
)

root_dir = "../"
data_dir = os.path.join(root_dir, "data")
log_dir = os.path.join(root_dir, "logs")

logger = WandbLogger(project="opf", save_dir=log_dir, config=params, tags=["test"])
barrier, trainer, dm = model_from_parameters(
    params,
    gpus=1,
    logger=logger,
    data_dir=data_dir,
    patience=params["patience"],
    eps=params["eps"],
    debug=False
)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [4]:
trainer.tune(barrier, datamodule=dm)
trainer.fit(barrier, dm)
trainer.test(datamodule=dm)
logger.finalize("finished")
wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                 | Type                | Params
-------------------------------------------------------------
0 | model                | GNN                 | 124 K 
1 | powerflow_parameters | PowerflowParameters | 0     
-------------------------------------------------------------
124 K     Trainable params
0         Non-trainable params
124 K     Total params
0.993     Total estimated model params size (MB)
Finding best initial lr:  98%|█████████▊| 98/100 [00:02<00:00, 37.31it/s]Restoring states from the checkpoint file at /home/damow/repos/OPF/notebooks/lr_find_temp_model.ckpt
Restored all states from the checkpoint file at /home/damow/repos/OPF/notebooks/lr_find_temp_model.ckpt
Finding best initial lr: 100%|██████████| 100/100 [00:02<00:00, 34.25it/s]
Learning rate set to 0.0002089296130854041
/home/damow/.pyenv/versions/3.9.5/envs/opf/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarnin


  | Name                 | Type                | Params
-------------------------------------------------------------
0 | model                | GNN                 | 124 K 
1 | powerflow_parameters | PowerflowParameters | 0     
-------------------------------------------------------------
124 K     Trainable params
0         Non-trainable params
124 K     Total params
0.993     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 392/392 [00:18<00:00, 15.77it/s, loss=233, v_num=5h82, train/loss=230.0, val/loss=230.0]

/home/damow/.pyenv/versions/3.9.5/envs/opf/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
/home/damow/.pyenv/versions/3.9.5/envs/opf/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Testing: 100%|██████████| 798/798 [00:29<00:00, 32.37it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 798/798 [00:29<00:00, 27.25it/s]


train/loss,238.23734
train/cost,0.06403
train/equality/loss,45.01468
train/equality/rate,1.0
train/equality/error_mean,1.1437
train/equality/error_max,8.49303
train/inequality/loss,193.22202
train/inequality/rate,0.32596
train/inequality/error_mean,1.37245
train/inequality/error_max,9.49049
epoch,0


train/loss,█▂▁▁▁▁▁
train/cost,▆██▇▅▃▁
train/equality/loss,█▂▁▁▁▁▁
train/equality/rate,▁▁▁▁▁▁▁
train/equality/error_mean,█▆▅▃▂▁▁
train/equality/error_max,█▃▁▁▄▅█
train/inequality/loss,▇██▇▅▃▁
train/inequality/rate,█▄▂▂▁▁▁
train/inequality/error_mean,█▂▁▁▁▁▂
train/inequality/error_max,▁██▇███
epoch,▁▁▁▁▁▁▁▁
